# Webscraping mit BeautifulSoup

In [1]:
# Modulimporte:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from PIL import Image

## Webscraping am Beispiel erklärt

Als Beispiel untersuchen wir die Webseite https://worldofwarcraft.com/de-de/game/classes

Um zu prüfen, was wir auf der Webseite dürfen und ob bestimmte Inhalte von Webscraping verboten sind, müssen wir die robots.txt Seite aufsuchen
https://worldofwarcraft.com/robots.txt

Den Inhalt der Webseite können wir schon in Python abrufen. Dazu können wir `requests.get` verwenden. Wir erhalten allerdings HTML-Code zurück, und müssen diesen durchsuchen, um an die Infos zu kommen, die uns interessieren. Dafür benutzen wir "Beautiful Soup 4".

In [2]:
# Schritt 1: Webseiten-Inhalte abrufen
url = 'https://worldofwarcraft.com/de-de/game/classes'
response = requests.get(url)
response.text

'<!DOCTYPE html><html lang="de-DE"><head><title>Spielbare Klassen</title><script>var optimizelyEnabled = false;\ntry {\n  optimizelyEnabled = JSON.parse(\'true\');\n} catch (err) {\n  console.log(err);\n}\n</script><script>var optimizelyLoaded = (function () {\n  var OPTIMIZELY_AGENT_LOADED_EVENT = \'OptimizelyWebLoaded\';\n  var OPTIMIZELY_FULLSTACK_DATAFILE_LOADED_EVENT = \'OptimizelyFullstackDataFileLoaded\';\n\n  function initOptimizely() {\n    var agentScript = document.createElement(\'script\');\n    agentScript.src = \'https://cdn.optimizely.com/js/8521175242.js\';\n    agentScript.onload = function () {\n      optimizelyLoaded = true;\n      trigger(OPTIMIZELY_AGENT_LOADED_EVENT);\n    };\n    document.head.appendChild(agentScript);\n\n    var optimizelySdkKey = \'\';\n    var optimizelySdkEnabled = false;\n    try {\n      optimizelySdkEnabled = JSON.parse(\'true\');\n    } catch (err) {\n      console.log(err);\n    }\n    if (optimizelySdkKey && optimizelySdkEnabled) {\n   

In [3]:
type(response.text)

str

In [4]:
# Schritt 2: Suppen-Objekt erstellen
# -> Inhalt wird von bs4 ausgelesen und umgewandelt
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# Schritt 3: Die Suppe durchsuchen
#            Welche HTML-Tags wollen wir finden?
titles = soup.find_all('div', class_='Card-title')
titles

[<div class="Card-title">Krieger</div>,
 <div class="Card-title">Jäger</div>,
 <div class="Card-title">Priester</div>,
 <div class="Card-title">Magier</div>,
 <div class="Card-title">Mönch</div>,
 <div class="Card-title">Dämonenjäger</div>,
 <div class="Card-title">Rufer</div>,
 <div class="Card-title">Paladin</div>,
 <div class="Card-title">Schurke</div>,
 <div class="Card-title">Schamane</div>,
 <div class="Card-title">Hexenmeister</div>,
 <div class="Card-title">Druide</div>,
 <div class="Card-title">Todesritter</div>]

In [6]:
len(titles)

13

In [7]:
# Prinzipiell hätte auch nur Klassenangabe gereicht:
soup.find_all(class_='Card-title')

[<div class="Card-title">Krieger</div>,
 <div class="Card-title">Jäger</div>,
 <div class="Card-title">Priester</div>,
 <div class="Card-title">Magier</div>,
 <div class="Card-title">Mönch</div>,
 <div class="Card-title">Dämonenjäger</div>,
 <div class="Card-title">Rufer</div>,
 <div class="Card-title">Paladin</div>,
 <div class="Card-title">Schurke</div>,
 <div class="Card-title">Schamane</div>,
 <div class="Card-title">Hexenmeister</div>,
 <div class="Card-title">Druide</div>,
 <div class="Card-title">Todesritter</div>]

In [8]:
# Schritt 4: Jetzt wollen wir für jedes Objekt nur
# den Textinhalt und diesen als neue Liste speichern.
classes = [title.text for title in titles]
classes

['Krieger',
 'Jäger',
 'Priester',
 'Magier',
 'Mönch',
 'Dämonenjäger',
 'Rufer',
 'Paladin',
 'Schurke',
 'Schamane',
 'Hexenmeister',
 'Druide',
 'Todesritter']

### Können wir jedes Attribut des HTML-Codes zum Filtern nutzen?
#### Klares Jein!

Abhängig vom Attributnamen müssen wir unterschiedlich vorgehen

In [9]:
# Kleiner Hinweis vorab: So kann man sich Attribute eines Tags anzeigen lassen.
# Bei class steht die Klasse oder die Klassen, denen ein Tag angehört
# Alle weiteren Schlüssel-Werte-Paare sind Attribute:
soup.a

<a class="Card-link SyncHeight-item" href="/de-de/game/classes/warrior"><div class="Card-image"><div class="Card-image--full"><div class="Art Card-art"><div class="Art-size" style="padding-top:100%"></div><div class="Art-image" style="background-image:url(https://blz-contentstack-images.akamaized.net/v3/assets/blt3452e3b114fab0cd/bltf9f62859db695a4d/5ee3e473a7c560086afc3ffd/9WL35DUZNSG01457032148687.jpg);"></div><div class="Art-overlay"></div></div></div><div class="Card-image--half"><div class="Art Card-art"><div class="Art-size" style="padding-top:50%"></div><div class="Art-image" style="background-image:url(https://blz-contentstack-images.akamaized.net/v3/assets/blt3452e3b114fab0cd/bltf9f62859db695a4d/5ee3e473a7c560086afc3ffd/9WL35DUZNSG01457032148687.jpg);"></div><div class="Art-overlay"></div></div></div></div><div class="Card-content"><div class="gutter-normal gutter-all"><div class="Card-title">Krieger</div><div class="Card-subtitle">Tank, Schaden</div><p class="Card-description

In [10]:
soup.a.attrs

{'class': ['Card-link', 'SyncHeight-item'],
 'href': '/de-de/game/classes/warrior'}

In [11]:
# Mal ein Element mit mehr Attributen:
soup.find(class_='Photoswipe--lightbox').attrs

{'class': ['Photoswipe', 'pswp', 'Photoswipe--lightbox', 'Photoswipe--wow'],
 'name': 'lightbox',
 'tabindex': '-1',
 'role': 'dialog',
 'aria-hidden': 'true'}

In [ ]:
# Wie können wir nun aber nach Attributen unseren HTML-Baum durchforsten?

In [12]:
# 1. Einfach: Übernahme als **kwargs
# Jeder Parameter, der nicht in der find_all-Methode
# definiert ist, wird als Attribut-Filter verwendet
# In diesem Fall href:
soup.find_all(name='a', class_='Card-link', href='/de-de/game/classes/warrior')

[<a class="Card-link SyncHeight-item" href="/de-de/game/classes/warrior"><div class="Card-image"><div class="Card-image--full"><div class="Art Card-art"><div class="Art-size" style="padding-top:100%"></div><div class="Art-image" style="background-image:url(https://blz-contentstack-images.akamaized.net/v3/assets/blt3452e3b114fab0cd/bltf9f62859db695a4d/5ee3e473a7c560086afc3ffd/9WL35DUZNSG01457032148687.jpg);"></div><div class="Art-overlay"></div></div></div><div class="Card-image--half"><div class="Art Card-art"><div class="Art-size" style="padding-top:50%"></div><div class="Art-image" style="background-image:url(https://blz-contentstack-images.akamaized.net/v3/assets/blt3452e3b114fab0cd/bltf9f62859db695a4d/5ee3e473a7c560086afc3ffd/9WL35DUZNSG01457032148687.jpg);"></div><div class="Art-overlay"></div></div></div></div><div class="Card-content"><div class="gutter-normal gutter-all"><div class="Card-title">Krieger</div><div class="Card-subtitle">Tank, Schaden</div><p class="Card-descriptio

In [13]:
# 2. Umständlicher: attrs-Argument
# Manche Attributnamen eignen sich nicht als
# Argumentname. Müssen als dict übergeben werden
# So geht das nicht:
# soup.find_all(media-large='font-size-xSmall')
soup.find_all(attrs={'media-large': 'font-size-xSmall'})

[<p class="Card-description font-size-xSmall" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen und Rüstungen, um in einer glorreichen Schlacht den Feind in den Untergang zu stürzen.</p>,
 <p class="Card-excerpt Card-excerpt--normal font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führun...</p>,
 <p class="Card-excerpt Card-excerpt--medium font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen und Rüstungen, um in einer glorreichen Schlac...</p>,
 <p class="Card-excerpt Card-excerpt-

In [14]:
# Und wie sieht es mit dem hier aus?
soup.find(class_='Photoswipe--lightbox').attrs

{'class': ['Photoswipe', 'pswp', 'Photoswipe--lightbox', 'Photoswipe--wow'],
 'name': 'lightbox',
 'tabindex': '-1',
 'role': 'dialog',
 'aria-hidden': 'true'}

In [16]:
soup.find(class_='Photoswipe--lightbox')

<div aria-hidden="true" class="Photoswipe pswp Photoswipe--lightbox Photoswipe--wow" name="lightbox" role="dialog" tabindex="-1"><div class="pswp__bg"></div><div class="pswp__scroll-wrap"><div class="pswp__container"><div class="pswp__item"></div><div class="pswp__item"></div><div class="pswp__item"></div></div><div class="pswp__ui pswp__ui--hidden"><div class="pswp__top-bar Photoswipe-top"><div class="pswp__counter"></div><button class="pswp__button pswp__button--close" title="Schließen (Esc)"></button><button class="pswp__button pswp__button--share" title="Teilen"></button><button class="pswp__button pswp__button--fs" title="Vollbild ein/aus"></button><button class="pswp__button pswp__button--zoom" title="Hereinzoomen/Herauszoomen"></button><div class="pswp__preloader"><div class="pswp__preloader__icn"><div class="pswp__preloader__cut"><div class="pswp__preloader__donut"></div></div></div></div></div><div class="pswp__share-modal pswp__share-modal--hidden pswp__single-tap"><div class

In [17]:
# Hier wäre nicht nur 'aria-hidden' problematisch, sondern auch 'name', weil das bereits
# für den Parameter Tagname vergeben ist!
# So funktioniert das nicht:
# soup.find_all(name='lightbox')
# Stattdessen:
soup_deluxe = soup.find_all(
    name='div',
    class_='Photoswipe--lightbox',
    attrs={'name': 'lightbox'},
)

soup_deluxe

[<div aria-hidden="true" class="Photoswipe pswp Photoswipe--lightbox Photoswipe--wow" name="lightbox" role="dialog" tabindex="-1"><div class="pswp__bg"></div><div class="pswp__scroll-wrap"><div class="pswp__container"><div class="pswp__item"></div><div class="pswp__item"></div><div class="pswp__item"></div></div><div class="pswp__ui pswp__ui--hidden"><div class="pswp__top-bar Photoswipe-top"><div class="pswp__counter"></div><button class="pswp__button pswp__button--close" title="Schließen (Esc)"></button><button class="pswp__button pswp__button--share" title="Teilen"></button><button class="pswp__button pswp__button--fs" title="Vollbild ein/aus"></button><button class="pswp__button pswp__button--zoom" title="Hereinzoomen/Herauszoomen"></button><div class="pswp__preloader"><div class="pswp__preloader__icn"><div class="pswp__preloader__cut"><div class="pswp__preloader__donut"></div></div></div></div></div><div class="pswp__share-modal pswp__share-modal--hidden pswp__single-tap"><div clas

In [18]:
len(soup_deluxe)

1

In [20]:
# 3. Nicht alles nutzbar
# Manche Attribute werden uns auf der Website zwar
# angezeigt, befinden sich aber gar nicht im 
# soup Objekt! 
soup.find('blz-nav').attrs

{'class': ['SiteNav'],
 'hidden': 'hidden',
 'user-endpoint': '/de-de/navbar/authenticate',
 'content': 'wow-site',
 'search-url': '/de-de/search?q={value}'}

In [21]:
soup.find('blz-nav').has_attr('queryselectoralways')

False

In [22]:
# .attrs gibt dict zurück
# --> Infos einzelner Attribute abrufbar
type(soup.find('blz-nav').attrs)

dict

In [24]:
attributes = soup.find('blz-nav').attrs
attributes

{'class': ['SiteNav'],
 'hidden': 'hidden',
 'user-endpoint': '/de-de/navbar/authenticate',
 'content': 'wow-site',
 'search-url': '/de-de/search?q={value}'}

In [26]:
attributes['search-url']

'/de-de/search?q={value}'

In [27]:
# Extraktion mehrerer Klassen auf einmal:
soup.find_all(class_=['Card-title', 'Card-subtitle', 'Card-excerpt--normal'])

[<div class="Card-title">Krieger</div>,
 <div class="Card-subtitle">Tank, Schaden</div>,
 <p class="Card-excerpt Card-excerpt--normal font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führun...</p>,
 <div class="Card-title">Jäger</div>,
 <div class="Card-subtitle">Schaden</div>,
 <p class="Card-excerpt Card-excerpt--normal font-size-small" media-large="font-size-xSmall">Schon seit ewigen Zeiten lockt der Ruf der Wildnis einige Abenteurer aus der Sicherheit ihres Heims hinaus in die gnadenlose, unberührte Welt. Di...</p>,
 <div class="Card-title">Priester</div>,
 <div class="Card-subtitle">Heiler, Schaden</div>,
 <p class="Card-excerpt Card-excerpt--normal font-size-small" media-large="font-size-xSmall">Priester haben sich ganz dem spirituellen Pfad gewidmet und zeigen ihren unerschütterlichen Glauben, indem sie anderen dienen. Vor Jahrtausenden ...</p

## Weitere Funktionen von Beautiful Soup

* In Kinder- oder Elternelemente navigieren
* Nach Textinhalt suchen
* Bestimmte Strukturen suchen (z.b. ein `<td>` innerhalb eines `<tr>`)

Suche nach Textinhalten

In [28]:
# Filtern mit str
# Holt Tag samt eingeschlossenen angegebenen Text heraus:
soup.find_all('div', string='Schurke')

[<div class="Card-title">Schurke</div>]

In [29]:
# Filtern mit str
# Funktioniert nicht als Teilstring
soup.find_all('div', string='Tank')  # 'Tank' == 'Tank, Schaden'

[]

In [30]:
# Regex als Suchparameter
# Pattern kann nicht direkt eingesetzt
# werden; Muss "kompiliert" werden.
soup.find_all('div', string=re.compile(r'Tank'))

[<div class="Card-subtitle">Tank, Schaden</div>,
 <div class="Card-subtitle">Tank, Heiler, Schaden</div>,
 <div class="Card-subtitle">Tank, Schaden</div>,
 <div class="Card-subtitle">Tank, Heiler, Schaden</div>,
 <div class="Card-subtitle">Tank, Heiler, Schaden</div>,
 <div class="Card-subtitle">Tank, Schaden</div>]

In [44]:
# Mini-Exkurs:
# re.compile erzeugt ein Regex-Objekt, das über verschiedene Methoden verfügt, die man 
# daran aufrufen kann
todes_regex = re.compile(r'\w*t[oö][dt]\w*', re.I)  # tot, Tod, töten, tödlich, abtöten

In [45]:
satz = 'Dass Urlaub töten kann... Ein tödlicher Unfall war das. Alle Passagiere fanden den Tod. Auch der Fahrer war tot. Ganz schön nervtötend.'

In [46]:
todes_regex.findall(satz)

['töten', 'tödlicher', 'Tod', 'tot', 'nervtötend']

In [ ]:
# Das heißt, find_all greift auf das kompilierte Regex-Objekt zu und ruft dann selber die entsprechende Methode auf.

In [ ]:
# Wie viele Paragraphen enthalten auf unserer Seite denn so etwas wie töten oder Tod?

In [48]:
todesparagraphen = soup.find_all('p', string=todes_regex)
todesparagraphen

[<p class="Card-description font-size-xSmall" media-large="font-size-xSmall">Raufbolde die mit bloßen Händen kämpfen haben diesen waffenlosen Stil im alten Pandaria entwickelt. Mönche die so verschieden sind, wie die Energie die sie umgibt. Sie können tödliche Angriffe durch den Dunst ihrer Gebräue schultern, den Chi-Fluß heilen und ihre Feinde mit Schlägen und Tritten so schnell wie der Wind eindecken.</p>,
 <p class="Card-excerpt Card-excerpt--medium font-size-small" media-large="font-size-xSmall">Raufbolde die mit bloßen Händen kämpfen haben diesen waffenlosen Stil im alten Pandaria entwickelt. Mönche die so verschieden sind, wie die Energie die sie umgibt. Sie können tödliche Angriffe durch den Dunst ihrer Gebräue schultern, den Chi-Flu...</p>,
 <p class="Card-excerpt Card-excerpt--long font-size-small" media-large="font-size-xSmall">Raufbolde die mit bloßen Händen kämpfen haben diesen waffenlosen Stil im alten Pandaria entwickelt. Mönche die so verschieden sind, wie die Energie di

In [49]:
len(todesparagraphen)

14

In [50]:
todesklassen = soup.find_all(class_='Card-title', string=todes_regex)
todesklassen

[<div class="Card-title">Todesritter</div>]

Eltern / Kind Navigation

In [51]:
# Infobox der ersten Klasse extrahieren
result = soup.find(class_='gutter-normal')
result

<div class="gutter-normal gutter-all"><div class="Card-title">Krieger</div><div class="Card-subtitle">Tank, Schaden</div><p class="Card-description font-size-xSmall" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen und Rüstungen, um in einer glorreichen Schlacht den Feind in den Untergang zu stürzen.</p><p class="Card-excerpt Card-excerpt--normal font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führun...</p><p class="Card-excerpt Card-excerpt--medium font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermesslic

In [52]:
len(result)

6

In [53]:
# Welche tags enthält mein Ergebnis (children)?
result.contents

[<div class="Card-title">Krieger</div>,
 <div class="Card-subtitle">Tank, Schaden</div>,
 <p class="Card-description font-size-xSmall" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen und Rüstungen, um in einer glorreichen Schlacht den Feind in den Untergang zu stürzen.</p>,
 <p class="Card-excerpt Card-excerpt--normal font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führun...</p>,
 <p class="Card-excerpt Card-excerpt--medium font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen un

In [54]:
type(result.contents)

list

In [55]:
len(result.contents)

6

In [56]:
# Children dagegen gibt uns einen Iterator, aber erzeugt keine Liste:
result.children

In [57]:
# ...lässt sich aber ohne Probleme in eine Liste packen...
list(result.children)

[<div class="Card-title">Krieger</div>,
 <div class="Card-subtitle">Tank, Schaden</div>,
 <p class="Card-description font-size-xSmall" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen und Rüstungen, um in einer glorreichen Schlacht den Feind in den Untergang zu stürzen.</p>,
 <p class="Card-excerpt Card-excerpt--normal font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führun...</p>,
 <p class="Card-excerpt Card-excerpt--medium font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen un

In [58]:
# Und ist bei einem Vergleich dann identisch mit contents:
result.contents == list(result.children)

True

In [62]:
children = list(result.children)

In [ ]:
# Es gibt auch noch das Attribut descendants, was alles meint, was von einem Tag abstammt

In [59]:
result.descendants

<generator object Tag.descendants at 0x000001DED1EDF5E0>

In [60]:
descendants = list(result.descendants)
descendants

[<div class="Card-title">Krieger</div>,
 'Krieger',
 <div class="Card-subtitle">Tank, Schaden</div>,
 'Tank, Schaden',
 <p class="Card-description font-size-xSmall" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen und Rüstungen, um in einer glorreichen Schlacht den Feind in den Untergang zu stürzen.</p>,
 'Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen und Rüstungen, um in einer glorreichen Schlacht den Feind in den Untergang zu stürzen.',
 <p class="Card-excerpt Card-excerpt--normal font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen si

In [63]:
# Die Länge der Kinder des Tags:
len(children)

6

In [64]:
len(descendants)

12

In [ ]:
# Quizfrage: Warum gibt es da unterschiedliche Zahlen?
# Es gibt i.d.R. immer mehr Nachfahren als direkte Kinder (auch in der Welt).
# Direkte Kinder sind Tags eine Ebene unter dem Elternteil.
# Nachkommen umfassen alle Ebenen.

In [65]:
for descendant in descendants:
    print(descendant)
    print()

<div class="Card-title">Krieger</div>

Krieger

<div class="Card-subtitle">Tank, Schaden</div>

Tank, Schaden

<p class="Card-description font-size-xSmall" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen und Rüstungen, um in einer glorreichen Schlacht den Feind in den Untergang zu stürzen.</p>

Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen und Rüstungen, um in einer glorreichen Schlacht den Feind in den Untergang zu stürzen.

<p class="Card-excerpt Card-excerpt--normal font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Fü

In [66]:
for child in children:
    print(child)
    print()

<div class="Card-title">Krieger</div>

<div class="Card-subtitle">Tank, Schaden</div>

<p class="Card-description font-size-xSmall" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen und Rüstungen, um in einer glorreichen Schlacht den Feind in den Untergang zu stürzen.</p>

<p class="Card-excerpt Card-excerpt--normal font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führun...</p>

<p class="Card-excerpt Card-excerpt--medium font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen und Rüs

In [73]:
# Einen "child"-Zweig betreten über den Index:
children[3]

<p class="Card-excerpt Card-excerpt--normal font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führun...</p>

In [75]:
# Umgekehrt: Welche Eltern hat mein Ergebnis?
result.parent

<div class="Card-content"><div class="gutter-normal gutter-all"><div class="Card-title">Krieger</div><div class="Card-subtitle">Tank, Schaden</div><p class="Card-description font-size-xSmall" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqualitäten und ein unermessliches Wissen über Waffen und Rüstungen, um in einer glorreichen Schlacht den Feind in den Untergang zu stürzen.</p><p class="Card-excerpt Card-excerpt--normal font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führun...</p><p class="Card-excerpt Card-excerpt--medium font-size-small" media-large="font-size-xSmall">Schon seit es Kriege gibt streben Helden aus allen Völkern danach, die Kunst der Schlacht zu beherrschen. Im Krieger vereinen sich Stärke, Führungsqual

#### Übung: Check TagesDeals!

Du interessierst dich dafür, welche Produkte in den Tagesdeals bei Alternate vorkommen. Link: https://www.alternate.de/TagesDeals


1. Prüfe , ob es erlaubt ist, Informationen von dieser Stelle zu extrahieren.
2. Identifiziere, wie sich die Produkte der heutigen Tagesdeals scrapen lassen. Welche Tags werden benutzt? Wie können wir die Elemente identifizieren?
3. Scrape die Daten und extrahiere die Infos, die dich interessieren.
4. Bringe den Output in ein passendes und lesbares Format (lege also z.B. einen ordentlich formatierten und benannten DataFrame an).


In [43]:
URL = 'https://www.alternate.de/TagesDeals'
website = requests.get(URL)
results = BeautifulSoup(website.content, 'html.parser')
deals = results.find_all(class_='product-name font-weight-bold')
print(deals)

[<div class="product-name font-weight-bold"><span class="mr-1">Enermax</span>Platimax D.F 1050W, PC-Netzteil</div>, <div class="product-name font-weight-bold"><span class="mr-1">Alpenföhn</span>Wing Boost 3 ARGB 140x140x25, Gehäuselüfter</div>, <div class="product-name font-weight-bold"><span class="mr-1">Acer</span>Nitro, Gaming-Maus</div>, <div class="product-name font-weight-bold"><span class="mr-1">Thermaltake</span>TOUGHLIQUID 280 ARGB Sync All-In-One Liquid Cooler 280mm, Wasserkühlung</div>, <div class="product-name font-weight-bold"><span class="mr-1">HP</span>Prelude 15,6, Rucksack</div>, <div class="product-name font-weight-bold"><span class="mr-1">Team Group</span>DIMM 32 GB DDR4-3600 (2x 16 GB) Dual-Kit, Arbeitsspeicher</div>, <div class="product-name font-weight-bold"><span class="mr-1">Seagate</span>Speichererweiterungskarte für Xbox Series X|S 2 TB, SSD</div>, <div class="product-name font-weight-bold"><span class="mr-1">Arctic</span>Liquid Freezer II 420 A-RGB 420mm, Was

In [44]:
deallist = []
for deal in deals:
    deallist.append(deal.text)

deallist.sort()
deallist

['AMDWraith STEALTH CPU-Kühler BULK',
 'AcerNitro, Gaming-Maus',
 'AcerPredator Aethon 301, Gaming-Tastatur',
 'AcerPredator Orion 7000 (DG.E39EG.005), Gaming-PC',
 'AcerSwift 3 (SF316-51-51SN), Notebook',
 'AcerSwift 3 Go (SFG14-71-51H2), Notebook',
 'AlpenföhnBrocken 4, CPU-Kühler',
 'AlpenföhnJetStream, Gehäuselüfter',
 'AlpenföhnWing Boost 3 ARGB 140x140x25, Gehäuselüfter',
 'AppleMacBook Pro (16") 2023, Notebook',
 'ArcticFreezer i35 A-RGB, CPU-Kühler',
 'ArcticLiquid Freezer II 420 A-RGB 420mm, Wasserkühlung',
 'CreativeOutlier Free Pro+, Kopfhörer',
 'CreativeOutlier Free Pro+, Kopfhörer',
 'EKWBEK-Nucleus AIO CR240 Lux D-RGB 240mm, Wasserkühlung',
 'EnermaxPlatimax D.F 1050W, PC-Netzteil',
 'EnermaxREVOLUTION D.F.X 1200W, PC-Netzteil',
 'EnermaxREVOLUTION D.F.X 850W, PC-Netzteil',
 'HP17-cn3157ng, Notebook',
 'HP17-cp2159ng, Notebook',
 'HP255 G9 (7N0S6ES), Notebook',
 'HP255 G9 (7N0S7ES), Notebook',
 'HP255 G9 (7N0S8ES), Notebook',
 'HP255 G9 (7N0T0ES), Notebook',
 'HPPrelude 

In [45]:
df = pd.DataFrame(deallist)
df = df[0].str.split(pat=r",([^,]+)$", expand = True, regex=True)
df.drop(columns=2, inplace=True)
df.columns = ["Produkt","Kategorie"]
df

,Produkt,Kategorie
0,AMDWraith STEALTH CPU-Kühler BULK,None
1,AcerNitro,Gaming-Maus
2,AcerPredator Aethon 301,Gaming-Tastatur
3,AcerPredator Orion 7000 (DG.E39EG.005),Gaming-PC
4,AcerSwift 3 (SF316-51-51SN),Notebook
5,AcerSwift 3 Go (SFG14-71-51H2),Notebook
6,AlpenföhnBrocken 4,CPU-Kühler
7,AlpenföhnJetStream,Gehäuselüfter
8,AlpenföhnWing Boost 3 ARGB 140x140x25,Gehäuselüfter
9,"AppleMacBook Pro (16"") 2023",Notebook


## Scrapen von Bildern

Wir wollen bei Wikipedia die Sternennacht von Van Gogh scrapen.

In [76]:
URL = 'https://de.wikipedia.org/wiki/Vincent_van_Gogh'
response = requests.get(URL)
soup = BeautifulSoup(response.text, 'html.parser')

In [77]:
# Abrufen aller Bildunterschriften
subtitles = soup.find_all('figcaption')
subtitles

[<figcaption><i>Selbstbildnis</i> (1887)<figure class="mw-default-size mw-halign-center skin-invert-image" typeof="mw:File/Frameless"><a class="mw-file-description" href="/wiki/Datei:Vincent_van_Gogh_signature_(Vincent).svg" title="Unterschrift von Vincent van Gogh"><img alt="Unterschrift von Vincent van Gogh" class="mw-file-element" data-file-height="168" data-file-width="433" decoding="async" height="85" src="//upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Vincent_van_Gogh_signature_%28Vincent%29.svg/220px-Vincent_van_Gogh_signature_%28Vincent%29.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Vincent_van_Gogh_signature_%28Vincent%29.svg/330px-Vincent_van_Gogh_signature_%28Vincent%29.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/c/cd/Vincent_van_Gogh_signature_%28Vincent%29.svg/440px-Vincent_van_Gogh_signature_%28Vincent%29.svg.png 2x" width="220"/></a><figcaption>Unterschrift von Vincent van Gogh</figcaption></figure></figcaption>,
 <figcaptio

In [78]:
# Soup-Element finden, das zur Sternennacht gehört
for subtitle in subtitles:
    if 'Sternennacht' in subtitle.text:
        starry_night = subtitle
        break

In [79]:
print(starry_night)

<figcaption><i><a href="/wiki/Sternennacht" title="Sternennacht">Sternennacht</a></i> (1889)</figcaption>


In [81]:
# Eine Ebene höher gehen, um Bild herauszubekommen
parent = starry_night.parent
parent

<figure class="mw-default-size" typeof="mw:File/Thumb"><a class="mw-file-description" href="/wiki/Datei:Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg"><img class="mw-file-element" data-file-height="23756" data-file-width="30000" decoding="async" height="174" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/220px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/330px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/440px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg 2x" width="220"/></a><figcaption><i><a href="/wiki/Sternennacht" title="Sternennacht">Sternennacht</a></i> (1889)</figcaption></figure>

In [82]:
image = parent.find('img')
image

<img class="mw-file-element" data-file-height="23756" data-file-width="30000" decoding="async" height="174" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/220px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/330px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/440px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg 2x" width="220"/>

In [83]:
# Welche Attribute hat mein Objekt?
image.attrs

{'src': '//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/220px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg',
 'decoding': 'async',
 'width': '220',
 'height': '174',
 'class': ['mw-file-element'],
 'srcset': '//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/330px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/440px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg 2x',
 'data-file-width': '30000',
 'data-file-height': '23756'}

In [84]:
image.attrs['src']

'//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/220px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg'

In [86]:
# Im Attribut "src" liegt der Pfad zum gespeicherten Bild
# Abrufbar mit .get
sublink = image.attrs.get('src')
sublink

'//upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/220px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg'

In [87]:
full_link = 'https:' + sublink
full_link

'https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg/220px-Van_Gogh_-_Starry_Night_-_Google_Art_Project.jpg'

In [88]:
img_response = requests.get(full_link)

In [91]:
image_data = img_response.content
image_data

b'\xff\xd8\xff\xdb\x00C\x00\x04\x03\x03\x04\x03\x03\x04\x04\x03\x04\x05\x04\x04\x05\x06\n\x07\x06\x06\x06\x06\r\t\n\x08\n\x0f\r\x10\x10\x0f\r\x0f\x0e\x11\x13\x18\x14\x11\x12\x17\x12\x0e\x0f\x15\x1c\x15\x17\x19\x19\x1b\x1b\x1b\x10\x14\x1d\x1f\x1d\x1a\x1f\x18\x1a\x1b\x1a\xff\xdb\x00C\x01\x04\x05\x05\x06\x05\x06\x0c\x07\x07\x0c\x1a\x11\x0f\x11\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\x1a\xff\xc0\x00\x11\x08\x00\xae\x00\xdc\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1d\x00\x00\x02\x03\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x05\x06\x03\x04\x07\x02\x08\x01\x00\t\xff\xc4\x00?\x10\x00\x02\x02\x01\x03\x03\x03\x02\x04\x05\x03\x03\x02\x04\x07\x00\x01\x02\x03\x04\x11\x05\x12!\x00\x061\x13"A\x14Q\x072a\x81\x15#Bq\x91Rb\xa1$3\xb1\x16C\x08\x17\xc1\xd1&Sr\x82\x92\xb2\xf0\xff\xc4\x00\x1b\x01\x00\x02\x03\x01\x01\x01\x00\x00\x00\x00

In [92]:
# Abgerufene Daten können wir dann speichern
with open('sternennacht.jpg', 'wb') as writer:
    writer.write(image_data)

In [93]:
# Anzeigen des neuen Bildes
image = Image.open('sternennacht.jpg')

In [94]:
image.show()